개선


In [ ]:
# 주요 라이브러리 및 모듈 가져오기
import torch  # 딥러닝 프레임워크 PyTorch
import torch.nn as nn  # 신경망 모듈
import torch.optim as optim  # 최적화 알고리즘
from torch.utils.data import DataLoader, TensorDataset  # 데이터 처리와 로딩 유틸리티
from sklearn.model_selection import train_test_split  # 데이터셋 분할 도구
from sklearn.preprocessing import StandardScaler, LabelEncoder  # 데이터 전처리 도구
import numpy as np  # 수학 연산
import pandas as pd  # 데이터 프레임 처리
from tqdm import tqdm  # 학습 진행 상태 표시
from sklearn.metrics import roc_auc_score, roc_curve  # 모델 성능 평가 지표
import matplotlib.pyplot as plt  # 데이터 시각화
from sklearn.datasets import fetch_openml  # 데이터셋 로드

In [ ]:
# 데이터 전처리 함수
def preprocess_dataframe(df):
    df_processed = df.copy()

    for column in df_processed.columns:
        if df_processed[column].dtype == 'category' or df_processed[column].dtype == 'object':
            if pd.api.types.is_categorical_dtype(df_processed[column]):
                df_processed[column] = df_processed[column].cat.add_categories(['Unknown'])
            df_processed[column] = df_processed[column].fillna('Unknown')
            le = LabelEncoder()
            df_processed[column] = le.fit_transform(df_processed[column].astype(str))

        elif df_processed[column].dtype in ['int64', 'float64']:
            df_processed[column] = df_processed[column].fillna(df_processed[column].median())

    return df_processed

In [ ]:
# 메인 함수 뜯기

# 데이터 로드 및 전처리
data = pd.read_csv("/content/sample_data/adult.csv",na_values=[' ?']).drop(['fnlwgt'], axis=1).dropna()
df_processed = preprocess_dataframe(data)

display(df_processed.head())

<ipython-input-35-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-35-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-35-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-35-150465cc5892>:7: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(df_processed[column]):
<ipython-input-35-150465

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,4,1,7,4,7,3,2,1,0,0,40,39,0
1,38,4,11,9,2,5,0,4,1,0,0,50,39,0
2,28,2,7,12,2,11,0,4,1,0,0,40,39,1
3,44,4,15,10,2,7,0,2,1,7688,0,40,39,1
4,18,0,15,10,4,0,3,4,0,0,0,30,39,0


In [ ]:
# 특성과 타겟, 민감한 특성 분리
sensitive_column = 'gender'  # 예시로 성별 선택
target_column = 'income'   # 예시로 타겟 컬럼 설정
X_data = df_processed.drop([target_column, sensitive_column], axis=1)
y_data = df_processed[target_column]
sensitive_feature = df_processed[sensitive_column]

display(X_data.head())
display(y_data.head())
display(sensitive_feature.head())

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country
0,25,4,1,7,4,7,3,2,0,0,40,39
1,38,4,11,9,2,5,0,4,0,0,50,39
2,28,2,7,12,2,11,0,4,0,0,40,39
3,44,4,15,10,2,7,0,2,7688,0,40,39
4,18,0,15,10,4,0,3,4,0,0,30,39


,income
0,0
1,0
2,1
3,1
4,0


,gender
0,1
1,1
2,1
3,1
4,0


In [ ]:
# 데이터 분할
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    X_data, y_data, sensitive_feature, test_size=0.2, random_state=42, stratify=y_data
)

display(X_train.head())
display(y_train.head())
display(sensitive_train.head())

,age,workclass,education,educational-num,marital-status,occupation,relationship,race,capital-gain,capital-loss,hours-per-week,native-country
34342,71,4,11,9,4,6,1,4,0,0,17,39
18559,17,4,0,6,4,12,2,4,0,0,10,39
12477,27,4,11,9,2,8,0,4,0,0,40,8
560,43,4,11,9,5,1,4,2,0,0,40,39
3427,31,4,9,13,2,4,0,4,0,0,40,39


,income
34342,0
18559,0
12477,0
560,0
3427,0


,gender
34342,1
18559,0
12477,1
560,0
3427,1


In [ ]:
# 데이터 표준화
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

display(X_train[:5]) # 12개 컬럼 ==> gender랑 fnlwgt 버려서
display(X_test[:5]) # 12개 컬럼 ==> gender랑 fnlwgt 버려서

array([[ 2.35103321,  0.09040954,  0.1854494 , -0.4193242 ,  0.91262793,
        -0.13431813, -0.27716485,  0.39071476, -0.14421841, -0.22013749,
        -1.8892569 ,  0.28754414],
       [-1.57914384,  0.09040954, -2.65604325, -1.58491023,  0.91262793,
         1.28629338,  0.34770747,  0.39071476, -0.14421841, -0.22013749,
        -2.45304549,  0.28754414],
       [-0.85133328,  0.09040954,  0.1854494 , -0.4193242 , -0.41234926,
         0.33921904, -0.90203718,  0.39071476, -0.14421841, -0.22013749,
        -0.03680866, -3.72595647],
       [ 0.31316363,  0.09040954,  0.1854494 , -0.4193242 ,  1.57511653,
        -1.31816106,  1.59745213, -1.98021205, -0.14421841, -0.22013749,
        -0.03680866,  0.28754414],
       [-0.56020905,  0.09040954, -0.33118562,  1.13479052, -0.41234926,
        -0.60785531, -0.90203718,  0.39071476, -0.14421841, -0.22013749,
        -0.03680866,  0.28754414]])

array([[ 1.11375525,  0.09040954,  0.1854494 , -0.4193242 , -0.41234926,
         0.33921904,  2.22232445, -1.98021205, -0.14421841, -0.22013749,
        -0.03680866,  0.28754414],
       [-0.77855222, -1.27721412,  0.1854494 , -0.4193242 ,  0.91262793,
        -1.31816106,  0.34770747,  0.39071476, -0.14421841, -0.22013749,
        -1.64763321,  0.28754414],
       [ 1.04097419, -1.27721412,  1.21871946, -0.03079552, -0.41234926,
        -0.60785531, -0.90203718,  0.39071476, -0.14421841, -0.22013749,
         0.76860362,  0.28754414],
       [ 1.40487947,  0.09040954,  0.1854494 , -0.4193242 , -0.41234926,
        -0.84462389, -0.90203718,  0.39071476,  0.28360815, -0.22013749,
        -1.96979812,  0.28754414],
       [ 0.60428785, -1.27721412, -0.84782065,  0.74626184, -0.41234926,
         1.04952479, -0.90203718,  0.39071476, -0.14421841, -0.22013749,
        -0.03680866,  0.28754414]])

In [ ]:
# PyTorch 텐서로 변환
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train.values)
y_test = torch.LongTensor(y_test.values)
sensitive_train = torch.LongTensor(sensitive_train.values)
sensitive_test = torch.LongTensor(sensitive_test.values)


print(X_train[:5])
print(y_train[:5])
print(sensitive_train[:5])

tensor([[ 2.3510,  0.0904,  0.1854, -0.4193,  0.9126, -0.1343, -0.2772,  0.3907,
         -0.1442, -0.2201, -1.8893,  0.2875],
        [-1.5791,  0.0904, -2.6560, -1.5849,  0.9126,  1.2863,  0.3477,  0.3907,
         -0.1442, -0.2201, -2.4530,  0.2875],
        [-0.8513,  0.0904,  0.1854, -0.4193, -0.4123,  0.3392, -0.9020,  0.3907,
         -0.1442, -0.2201, -0.0368, -3.7260],
        [ 0.3132,  0.0904,  0.1854, -0.4193,  1.5751, -1.3182,  1.5975, -1.9802,
         -0.1442, -0.2201, -0.0368,  0.2875],
        [-0.5602,  0.0904, -0.3312,  1.1348, -0.4123, -0.6079, -0.9020,  0.3907,
         -0.1442, -0.2201, -0.0368,  0.2875]])
tensor([0, 0, 0, 0, 0])
tensor([1, 0, 1, 0, 1])


In [ ]:
# DataLoader 생성
train_dataset = TensorDataset(X_train, y_train, sensitive_train)
test_dataset = TensorDataset(X_test, y_test, sensitive_test)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:

# 통합된 ColumnWiseInteraction 클래스
class ColumnWiseInteraction(nn.Module):
    def __init__(self, input_dim, interaction_dim):
        super().__init__()
        # 명시적인 행렬 곱을 위한 가중치 파라미터
        self.interaction_weights = nn.Parameter(torch.randn(input_dim, interaction_dim))

        # 학습 가능한 비선형 상호작용을 위한 모듈 정의
        self.linear1 = nn.Linear(input_dim, interaction_dim)  # 첫 번째 선형 변환
        self.activation = nn.ReLU()  # 비선형 활성화 함수
        self.linear2 = nn.Linear(interaction_dim, input_dim)  # 두 번째 선형 변환

    def forward(self, x):
        # 명시적인 컬럼 간 상호작용 (matmul 기반)
        matmul_interactions = torch.matmul(x, self.interaction_weights)

        # 학습 가능한 비선형 상호작용
        nonlinear_interactions = self.linear1(x)
        nonlinear_interactions = self.activation(nonlinear_interactions)
        nonlinear_interactions = self.linear2(nonlinear_interactions)

        # 원본 입력, matmul 상호작용, 비선형 상호작용 결합
        return torch.cat([x, matmul_interactions, nonlinear_interactions], dim=1)

In [ ]:
interaction_layer = ColumnWiseInteraction(input_dim=12, interaction_dim=32)
output = interaction_layer(X_train)

print("입력 크기:", X_train.shape)       # (5, 12)
print("출력 크기:", output.shape)

print(output[:5])

입력 크기: torch.Size([39073, 12])
출력 크기: torch.Size([39073, 56])
tensor([[ 2.3510e+00,  9.0410e-02,  1.8545e-01, -4.1932e-01,  9.1263e-01,
         -1.3432e-01, -2.7716e-01,  3.9071e-01, -1.4422e-01, -2.2014e-01,
         -1.8893e+00,  2.8754e-01,  4.9505e+00,  1.8490e+00,  1.9813e+00,
         -5.0161e-01, -2.2248e-01,  6.4609e+00,  4.2333e-01,  1.6862e-01,
         -1.9124e+00,  2.0753e+00, -1.3277e+00,  2.5257e+00, -3.0020e+00,
         -1.3081e+00,  4.3539e-01, -2.4231e+00, -1.0847e+00, -2.0944e+00,
         -5.3970e+00, -1.4622e+00,  4.7693e-01,  1.7055e+00, -2.0818e+00,
          9.4799e-01,  6.4581e+00, -2.0009e+00,  4.6550e+00,  1.0501e+00,
          3.6867e+00,  3.1338e+00,  4.5441e-02,  1.3111e+00, -1.3791e-01,
          4.1183e-01,  3.8449e-01, -3.2960e-01,  2.2030e-01,  2.8259e-01,
          5.4416e-02,  4.3199e-02, -1.3163e-01, -2.8003e-02, -1.0738e-01,
         -3.5434e-02],
        [-1.5791e+00,  9.0410e-02, -2.6560e+00, -1.5849e+00,  9.1263e-01,
          1.2863e+00,  3.47

In [ ]:
# 공정성 지표 계산을 위한 클래스 정의
class FairnessMetrics:
    @staticmethod
    def equal_opportunity_difference(predictions, true_labels, sensitive_features, target_label=1, epsilon=1e-6):
        """
        True Positive Rate(TPR)의 민감한 그룹 간 차이를 계산.
        """
        mask_positive = (true_labels == target_label)  # 실제 레이블이 positive인 경우 필터
        mask_sensitive_0 = (sensitive_features == 0) & mask_positive  # 그룹 0의 positive 필터
        mask_sensitive_1 = (sensitive_features == 1) & mask_positive  # 그룹 1의 positive 필터

        # 각 그룹의 TPR 계산
        tpr_0 = torch.mean(predictions[mask_sensitive_0]) if mask_sensitive_0.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        tpr_1 = torch.mean(predictions[mask_sensitive_1]) if mask_sensitive_1.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        return torch.abs(tpr_0 - tpr_1)  # TPR 차이 반환

    @staticmethod
    def false_positive_rate_difference(predictions, true_labels, sensitive_features, target_label=0, epsilon=1e-6):
        """
        False Positive Rate(FPR)의 민감한 그룹 간 차이를 계산.
        """
        mask_negative = (true_labels == target_label)  # 실제 레이블이 negative인 경우 필터
        mask_sensitive_0 = (sensitive_features == 0) & mask_negative
        mask_sensitive_1 = (sensitive_features == 1) & mask_negative

        # 각 그룹의 FPR 계산
        fpr_0 = torch.mean(predictions[mask_sensitive_0]) if mask_sensitive_0.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        fpr_1 = torch.mean(predictions[mask_sensitive_1]) if mask_sensitive_1.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        return torch.abs(fpr_0 - fpr_1)

    @staticmethod
    def equalized_odds_difference(predictions, true_labels, sensitive_features, target_label=1, epsilon=1e-6):
        """
        Equalized Odds 차이를 계산: EOP와 FPR 차이 중 최대값.
        """
        eop = FairnessMetrics.equal_opportunity_difference(predictions, true_labels, sensitive_features, target_label, epsilon) # True Positive Rate(TPR)의 민감한 그룹 간 차이
        fpr_diff = FairnessMetrics.false_positive_rate_difference(predictions, true_labels, sensitive_features, target_label=0, epsilon=epsilon) # False Positive Rate(FPR)의 민감한 그룹 간 차이
        return torch.max(eop, fpr_diff)  # 두 지표 중 큰 값을 반환

    @staticmethod
    def demographic_parity_difference(predictions, sensitive_features, epsilon=1e-6):
        """
        Positive 예측률의 민감한 그룹 간 차이를 계산.
        """
        mask_sensitive_0 = (sensitive_features == 0)
        mask_sensitive_1 = (sensitive_features == 1)

        rate_0 = torch.mean(predictions[mask_sensitive_0]) if mask_sensitive_0.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        rate_1 = torch.mean(predictions[mask_sensitive_1]) if mask_sensitive_1.sum() > 0 else torch.tensor(0.0).to(predictions.device)
        return torch.abs(rate_0 - rate_1)  # 차이 반환

In [ ]:
# 공정성을 고려한 Transformer 분류기 정의
class FairnessAwareTransformerClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_classes, interaction_dim=32):
        super().__init__()
        self.column_interaction = ColumnWiseInteraction(input_dim, interaction_dim)
        self.input_projection = nn.Linear(1, hidden_dim)  # 각 특성을 임베딩
        self.hidden_dim = hidden_dim  # Hidden dimension 저장
        self.num_heads = num_heads  # Transformer 헤드 수
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim, nhead=num_heads, dim_feedforward=hidden_dim * 4, batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=2)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, x):
        # Column-wise interaction 적용
        x = self.column_interaction(x)
        seq_len = x.size(1)  # 입력의 sequence 길이 (특성 수)

        # Input projection 및 차원 확장
        x = x.unsqueeze(-1)  # (batch_size, seq_len, 1)
        x = self.input_projection(x)  # (batch_size, seq_len, hidden_dim)

        # 동적으로 위치 인코딩 생성
        positional_encoding = torch.randn(1, seq_len, self.hidden_dim, device=x.device)
        x = x + positional_encoding

        # Transformer Encoder 적용
        x = self.transformer_encoder(x)

        # Sequence의 평균을 사용하여 분류 입력 생성
        x = x.mean(dim=1)
        return self.classifier(x)

In [ ]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.01):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 손실 합산 (가중치를 조정할 수 있습니다)
        fairness_loss = eop + eo + dp

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

In [ ]:
# 평가 함수
def evaluate_model(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    all_predictions = []
    all_labels = []
    all_sensitive = []

    with torch.no_grad():
        for batch_x, batch_y, batch_sensitive in test_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()

            # 공정성 지표 계산을 위한 데이터 수집
            probabilities = torch.softmax(outputs, dim=1)[:, 1]
            all_predictions.append(probabilities.cpu())
            all_labels.append(batch_y.cpu())
            all_sensitive.append(batch_sensitive.cpu())

    accuracy = 100 * correct / total

    # 모든 배치를 하나로 합침
    all_predictions = torch.cat(all_predictions)
    all_labels = torch.cat(all_labels)
    all_sensitive = torch.cat(all_sensitive)

    # 공정성 지표 계산
    eop = FairnessMetrics.equal_opportunity_difference(all_predictions, all_labels, all_sensitive, target_label=1).item()
    eo = FairnessMetrics.equalized_odds_difference(all_predictions, all_labels, all_sensitive, target_label=1, epsilon=1e-6).item()
    dp = FairnessMetrics.demographic_parity_difference(all_predictions, all_sensitive).item()

    return accuracy, eop, eo, dp

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}')


Training:  10%|█         | 1/10 [00:07<01:08,  7.66s/it]

Epoch [1/10], Loss: 0.4621, Fairness Loss: 0.3182, EOP: 0.1165, EO: 0.1272, DP: 0.0745, Test Accuracy: 79.91%, Test EOP: 0.0618, Test EO: 0.0671, Test DP: 0.1075


Training:  20%|██        | 2/10 [00:15<01:00,  7.50s/it]

Epoch [2/10], Loss: 0.4103, Fairness Loss: 0.3706, EOP: 0.1249, EO: 0.1432, DP: 0.1026, Test Accuracy: 77.53%, Test EOP: 0.0391, Test EO: 0.0562, Test DP: 0.0935


Training:  30%|███       | 3/10 [00:23<00:55,  7.93s/it]

...

Epoch [8/10], Loss: 0.3719, Fairness Loss: 0.4290, EOP: 0.1436, EO: 0.1622, DP: 0.1232, Test Accuracy: 83.56%, Test EOP: 0.0422, Test EO: 0.0714, Test DP: 0.1273


Training:  90%|█████████ | 9/10 [01:07<00:07,  7.43s/it]

Epoch [9/10], Loss: 0.3680, Fairness Loss: 0.4407, EOP: 0.1487, EO: 0.1672, DP: 0.1248, Test Accuracy: 83.10%, Test EOP: 0.0331, Test EO: 0.0712, Test DP: 0.1262


Training: 100%|██████████| 10/10 [01:15<00:00,  7.58s/it]

Epoch [10/10], Loss: 0.3748, Fairness Loss: 0.4336, EOP: 0.1453, EO: 0.1644, DP: 0.1239, Test Accuracy: 82.94%, Test EOP: 0.0289, Test EO: 0.0813, Test DP: 0.1335


# lambda_fairness=0.1

In [ ]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.1):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 손실 합산 (가중치를 조정할 수 있습니다)
        fairness_loss = eop + eo + dp

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}')


Training:  10%|█         | 1/10 [00:07<01:08,  7.61s/it]

Epoch [1/10], Loss: 0.4354, Fairness Loss: 0.3452, EOP: 0.1297, EO: 0.1371, DP: 0.0783, Test Accuracy: 82.66%, Test EOP: 0.0422, Test EO: 0.0422, Test DP: 0.0915


Training:  20%|██        | 2/10 [00:14<00:58,  7.34s/it]

Epoch [2/10], Loss: 0.3848, Fairness Loss: 0.4165, EOP: 0.1501, EO: 0.1611, DP: 0.1053, Test Accuracy: 83.95%, Test EOP: 0.0343, Test EO: 0.0620, Test DP: 0.1212


Training:  30%|███       | 3/10 [00:22<00:51,  7.33s/it]

...

Epoch [8/10], Loss: 0.3582, Fairness Loss: 0.4952, EOP: 0.1737, EO: 0.1894, DP: 0.1321, Test Accuracy: 83.99%, Test EOP: 0.0509, Test EO: 0.0745, Test DP: 0.1342


Training:  90%|█████████ | 9/10 [01:07<00:07,  7.46s/it]

Epoch [9/10], Loss: 0.3610, Fairness Loss: 0.4633, EOP: 0.1667, EO: 0.1781, DP: 0.1184, Test Accuracy: 84.10%, Test EOP: 0.0048, Test EO: 0.0536, Test DP: 0.1154


Training: 100%|██████████| 10/10 [01:13<00:00,  7.40s/it]

Epoch [10/10], Loss: 0.3659, Fairness Loss: 0.4667, EOP: 0.1720, EO: 0.1824, DP: 0.1122, Test Accuracy: 83.86%, Test EOP: 0.0044, Test EO: 0.0538, Test DP: 0.1128


## fairness_lambda = 0.05

In [ ]:
# 학습 함수
def train_model(model, train_loader, criterion, optimizer, device, lambda_fairness=0.05):
    model.train()
    total_loss = 0
    total_fairness_loss = 0
    total_eop = 0
    total_eo = 0
    total_dp = 0
    total_batches = 0

    for batch_x, batch_y, batch_sensitive in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        batch_sensitive = batch_sensitive.to(device)

        optimizer.zero_grad()

        # 예측 및 분류 손실
        outputs = model(batch_x)
        classification_loss = criterion(outputs, batch_y)

        # 공정성 손실 계산
        probabilities = torch.softmax(outputs, dim=1)[:, 1]  # Probability of class 1

        # To avoid NaNs, ensure that probabilities are between 0 and 1
        probabilities = torch.clamp(probabilities, min=0.0, max=1.0)

        # 공정성 지표 계산
        eop = FairnessMetrics.equal_opportunity_difference(probabilities, batch_y, batch_sensitive, target_label=1)
        eo = FairnessMetrics.equalized_odds_difference(probabilities, batch_y, batch_sensitive, target_label=1, epsilon=1e-6)
        dp = FairnessMetrics.demographic_parity_difference(probabilities, batch_sensitive)

        # 공정성 손실 합산 (가중치를 조정할 수 있습니다)
        fairness_loss = eop + eo + dp

        # 전체 손실 결합
        total_loss_batch = classification_loss + lambda_fairness * fairness_loss
        total_loss_batch.backward()
        optimizer.step()

        # 손실 및 공정성 지표 기록
        total_loss += classification_loss.item()
        total_fairness_loss += fairness_loss.item()
        total_eop += eop.item()
        total_eo += eo.item()
        total_dp += dp.item()
        total_batches += 1

    avg_loss = total_loss / total_batches
    avg_fairness_loss = total_fairness_loss / total_batches
    avg_eop = total_eop / total_batches
    avg_eo = total_eo / total_batches
    avg_dp = total_dp / total_batches

    return avg_loss, avg_fairness_loss, avg_eop, avg_eo, avg_dp

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 초기화
input_dim = X_train.shape[1]
hidden_dim = 128
num_heads = 4
num_classes = 2

model = FairnessAwareTransformerClassifier(input_dim, hidden_dim, num_heads, num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10  # 충분한 학습을 위해 epoch 수 증가

# 기록을 위한 리스트 초기화
train_fairness_history = {
    'loss': [],
    'fairness_loss': [],
    'EOP': [],
    'EO': [],
    'DP': []
}
test_fairness_history = {
    'accuracy': [],
    'EOP': [],
    'EO': [],
    'DP': []
}

for epoch in tqdm(range(num_epochs), desc="Training"):
    train_loss, train_fairness_loss, train_eop, train_eo, train_dp = train_model(
        model, train_loader, criterion, optimizer,
        device, lambda_fairness=0.1
    )
    test_acc, test_eop, test_eo, test_dp = evaluate_model(model, test_loader, device)

    # 기록 저장
    train_fairness_history['loss'].append(train_loss)
    train_fairness_history['fairness_loss'].append(train_fairness_loss)
    train_fairness_history['EOP'].append(train_eop)
    train_fairness_history['EO'].append(train_eo)
    train_fairness_history['DP'].append(train_dp)

    test_fairness_history['accuracy'].append(test_acc)
    test_fairness_history['EOP'].append(test_eop)
    test_fairness_history['EO'].append(test_eo)
    test_fairness_history['DP'].append(test_dp)

    print(f'Epoch [{epoch+1}/{num_epochs}], '
          f'Loss: {train_loss:.4f}, '
          f'Fairness Loss: {train_fairness_loss:.4f}, '
          f'EOP: {train_eop:.4f}, '
          f'EO: {train_eo:.4f}, '
          f'DP: {train_dp:.4f}, '
          f'Test Accuracy: {test_acc:.2f}%, '
          f'Test EOP: {test_eop:.4f}, '
          f'Test EO: {test_eo:.4f}, '
          f'Test DP: {test_dp:.4f}')


Training:  10%|█         | 1/10 [00:07<01:08,  7.65s/it]

Epoch [1/10], Loss: 0.4498, Fairness Loss: 0.3256, EOP: 0.1239, EO: 0.1317, DP: 0.0700, Test Accuracy: 79.63%, Test EOP: 0.0430, Test EO: 0.0438, Test DP: 0.0841


Training:  20%|██        | 2/10 [00:14<00:57,  7.18s/it]

Epoch [2/10], Loss: 0.4520, Fairness Loss: 0.3307, EOP: 0.1309, EO: 0.1362, DP: 0.0637, Test Accuracy: 78.81%, Test EOP: 0.0161, Test EO: 0.0161, Test DP: 0.0360


Training:  30%|███       | 3/10 [00:22<00:51,  7.38s/it]

...

Epoch [8/10], Loss: 0.4522, Fairness Loss: 0.3219, EOP: 0.1311, EO: 0.1357, DP: 0.0551, Test Accuracy: 80.02%, Test EOP: 0.0226, Test EO: 0.0226, Test DP: 0.0541


Training:  90%|█████████ | 9/10 [01:05<00:07,  7.27s/it]

Epoch [9/10], Loss: 0.4424, Fairness Loss: 0.3313, EOP: 0.1331, EO: 0.1381, DP: 0.0600, Test Accuracy: 80.03%, Test EOP: 0.0306, Test EO: 0.0306, Test DP: 0.0587


Training: 100%|██████████| 10/10 [01:13<00:00,  7.36s/it]

Epoch [10/10], Loss: 0.4407, Fairness Loss: 0.3193, EOP: 0.1262, EO: 0.1326, DP: 0.0605, Test Accuracy: 79.97%, Test EOP: 0.0292, Test EO: 0.0292, Test DP: 0.0570
